<a href="https://colab.research.google.com/github/kirank981/Project_space/blob/main/project_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependences


Install the necessary packages for PyTorch (torch and torchvision) and Flower (flwr) and pandas

In [1]:
!pip install -q flwr[simulation] torch torchvision matplotlib pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 34.7 MB/s eta 0:00:00


Import everything we need

In [2]:
from collections import OrderedDict
from typing import List, Tuple

from google.colab import drive
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl
from flwr.common import Metrics

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.0.1+cu118 and Flower 1.4.0


# Loading the data

Mounting drive

In [3]:
# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Setting the path to the location of the file

In [4]:
# Define the path to daily dataset folder
daily_dataset_path = Path('/content/drive/MyDrive/Federated learning implementation/dataset/dataset_archive/daily_dataset/daily_dataset')

# Define the path to daily dataset folder
weather_daily_dataset_path = Path('/content/drive/MyDrive/Federated learning implementation/dataset/dataset_archive/weather_daily_dataset.csv')


## Loading daily data
(of energy consumption)


In [5]:

# Initializing list to store dataframes
dfs = []

# Loop through the CSV files and reading them into dataframes
for i in range(111):
    filename = f'block_{i}.csv'
    df = pd.read_csv(daily_dataset_path / filename)
    dfs.append(df)

# Concatenating all the dataframes into a single dataframe
energy_daily_data = pd.concat(dfs, ignore_index=True)

# NRATM
# # Group the data by LCLid and create a dictionary of dataframes
# grouped_data = dict(tuple(energy_daily_data.groupby('LCLid')))


Loading data using file name

In [6]:
# Loading data from a specific CSV file
specific_file_data = pd.read_csv(daily_dataset_path / 'block_0.csv')

# Displaying data
print(specific_file_data)

           LCLid         day  energy_median  energy_mean  energy_max  \
0      MAC000002  2012-10-12         0.1385     0.154304       0.886   
1      MAC000002  2012-10-13         0.1800     0.230979       0.933   
2      MAC000002  2012-10-14         0.1580     0.275479       1.085   
3      MAC000002  2012-10-15         0.1310     0.213688       1.164   
4      MAC000002  2012-10-16         0.1450     0.203521       0.991   
...          ...         ...            ...          ...         ...   
25569  MAC005492  2014-02-24         0.1690     0.175042       0.378   
25570  MAC005492  2014-02-25         0.1550     0.160792       0.545   
25571  MAC005492  2014-02-26         0.1490     0.178542       0.687   
25572  MAC005492  2014-02-27         0.1140     0.146167       0.478   
25573  MAC005492  2014-02-28         0.0880     0.088000       0.088   

       energy_count  energy_std  energy_sum  energy_min  
0                46    0.196034       7.098       0.000  
1                48

### Loading required data
Creating a DataFrame that have only the required data

In [7]:
selected_column = ['LCLid','energy_sum','day']
energy_daily_selected=energy_daily_data[selected_column]
print(energy_daily_selected)

# Group the data by LCLid and create a dictionary of dataframes
grouped_data_selected = dict(tuple(energy_daily_selected.groupby('LCLid')))


             LCLid  energy_sum         day
0        MAC000002       7.098  2012-10-12
1        MAC000002      11.087  2012-10-13
2        MAC000002      13.223  2012-10-14
3        MAC000002      10.257  2012-10-15
4        MAC000002       9.769  2012-10-16
...            ...         ...         ...
3500520  MAC005567       4.107  2014-02-24
3500521  MAC005567       5.762  2014-02-25
3500522  MAC005567       5.066  2014-02-26
3500523  MAC005567       3.217  2014-02-27
3500524  MAC005567       0.183  2014-02-28

[3500525 rows x 3 columns]


Loading data using the 'LCLid'

In [8]:
# Now can access each dataframe separately using the LCLid as the key
# for "MAC000002"(random)
mac000002_data = grouped_data_selected['MAC000002']

# Display the data for 'MAC000002'
print(mac000002_data)

         LCLid  energy_sum         day
0    MAC000002       7.098  2012-10-12
1    MAC000002      11.087  2012-10-13
2    MAC000002      13.223  2012-10-14
3    MAC000002      10.257  2012-10-15
4    MAC000002       9.769  2012-10-16
..         ...         ...         ...
500  MAC000002      12.528  2014-02-24
501  MAC000002      11.826  2014-02-25
502  MAC000002      12.328  2014-02-26
503  MAC000002      20.518  2014-02-27
504  MAC000002       1.387  2014-02-28

[505 rows x 3 columns]


## Loading daily weather data

Creating a 'day' column that stores only the date values from 'time' column
(for linking weather dataset 'day' with daily dataset 'day')

In [30]:
# Load the weather dataset into a DataFrame
weather_daily_data = pd.read_csv(weather_daily_dataset_path)

# Convert the 'time' column to datetime format
weather_daily_data['time'] = pd.to_datetime(weather_daily_data['time'])

# Calculate the mean temperature for each day and store it in a new column 'mean_temp'
weather_daily_data['mean_temp'] = (weather_daily_data['temperatureMax'] + weather_daily_data['temperatureMin']) / 2

# Print the updated DataFrame
print(weather_daily_data)

     temperatureMax   temperatureMaxTime  windBearing                 icon  \
0             11.96  2011-11-11 23:00:00          123                  fog   
1              8.59  2011-12-11 14:00:00          198    partly-cloudy-day   
2             10.33  2011-12-27 02:00:00          225    partly-cloudy-day   
3              8.07  2011-12-02 23:00:00          232                 wind   
4              8.22  2011-12-24 23:00:00          252  partly-cloudy-night   
..              ...                  ...          ...                  ...   
877            9.03  2014-01-26 16:00:00          233    partly-cloudy-day   
878           10.31  2014-02-27 14:00:00          224    partly-cloudy-day   
879           18.97  2014-03-09 14:00:00          172  partly-cloudy-night   
880            8.83  2014-02-12 16:00:00          210                 wind   
881            9.90  2014-02-15 12:00:00          233                 wind   

     dewPoint   temperatureMinTime  cloudCover  windSpeed  pres

### Loading required data
Creating a DataFrame that have only the required data

In [31]:
# Create a new DataFrame with selected columns
selected_columns = ['mean_temp', 'pressure', 'humidity', 'windSpeed', 'time']
weather_selected = weather_daily_data[selected_columns]

# Print the new dataset
print(weather_selected)

     mean_temp  pressure  humidity  windSpeed       time
0       10.405   1016.08      0.95       3.88 2011-11-11
1        5.535   1007.71      0.88       3.94 2011-12-11
2        9.180   1032.76      0.74       3.54 2011-12-27
3        5.315   1012.12      0.87       3.00 2011-12-02
4        5.695   1028.17      0.80       4.46 2011-12-24
..         ...       ...       ...        ...        ...
877      6.145   1002.10      0.79       4.55 2014-01-26
878      7.120   1007.02      0.74       4.14 2014-02-27
879     13.310   1022.44      0.58       2.78 2014-03-09
880      5.930    994.27      0.75       7.24 2014-02-12
881      7.640    988.63      0.69       9.96 2014-02-15

[882 rows x 5 columns]


## Combining the datasets with features and target column

In [26]:
print("Columns in energy_daily_selected:", energy_daily_selected.columns)
print("Columns in weather_selected:", weather_selected.columns)


Columns in energy_daily_selected: Index(['LCLid', 'energy_sum', 'day'], dtype='object')
Columns in weather_selected: Index(['mean_temp', 'pressure', 'humidity', 'windSpeed', 'time'], dtype='object')


In [32]:
# to combine the datasets, create a new dataset that will copy data from the household and then add the weather data.
# After this, the data with non-matching dates between weather and household dataset will be considered missing values and removed

# Create a new DataFrame using energy_daily_selected
new_dataset = weather_selected.copy()

# Add new columns with 'NA' values
new_columns = ['LCLid','energy_sum','day']
for column in new_columns:
    new_dataset[column] = np.nan

# Print the new dataset
print(new_dataset)


     mean_temp  pressure  humidity  windSpeed       time  LCLid  energy_sum  \
0       10.405   1016.08      0.95       3.88 2011-11-11    NaN         NaN   
1        5.535   1007.71      0.88       3.94 2011-12-11    NaN         NaN   
2        9.180   1032.76      0.74       3.54 2011-12-27    NaN         NaN   
3        5.315   1012.12      0.87       3.00 2011-12-02    NaN         NaN   
4        5.695   1028.17      0.80       4.46 2011-12-24    NaN         NaN   
..         ...       ...       ...        ...        ...    ...         ...   
877      6.145   1002.10      0.79       4.55 2014-01-26    NaN         NaN   
878      7.120   1007.02      0.74       4.14 2014-02-27    NaN         NaN   
879     13.310   1022.44      0.58       2.78 2014-03-09    NaN         NaN   
880      5.930    994.27      0.75       7.24 2014-02-12    NaN         NaN   
881      7.640    988.63      0.69       9.96 2014-02-15    NaN         NaN   

     day  
0    NaN  
1    NaN  
2    NaN  
3    Na

In [40]:


# Add a new column with only the date value
new_dataset['date_only'] = new_dataset['time'].dt.date

# Print the updated weather_selected DataFrame
print(new_dataset)


     mean_temp  pressure  humidity  windSpeed       time  LCLid  energy_sum  \
0       10.405   1016.08      0.95       3.88 2011-11-11    NaN         NaN   
1        5.535   1007.71      0.88       3.94 2011-12-11    NaN         NaN   
2        9.180   1032.76      0.74       3.54 2011-12-27    NaN         NaN   
3        5.315   1012.12      0.87       3.00 2011-12-02    NaN         NaN   
4        5.695   1028.17      0.80       4.46 2011-12-24    NaN         NaN   
..         ...       ...       ...        ...        ...    ...         ...   
877      6.145   1002.10      0.79       4.55 2014-01-26    NaN         NaN   
878      7.120   1007.02      0.74       4.14 2014-02-27    NaN         NaN   
879     13.310   1022.44      0.58       2.78 2014-03-09    NaN         NaN   
880      5.930    994.27      0.75       7.24 2014-02-12    NaN         NaN   
881      7.640    988.63      0.69       9.96 2014-02-15    NaN         NaN   

     day   date_only  
0    NaN  2011-11-11  
1    

In [43]:

# # Iterate through rows in new_dataset
# for index, row in new_dataset.iterrows():
#     day = row['day']
#     matching_weather = weather_selected[weather_selected['date_only'] == day]
   # matching_weather = weather_selected[weather_selected['time'].dt.date == day]

#     # If matching weather data is found, update columns in new_dataset
#     if not matching_weather.empty:
#         matching_weather_row = matching_weather.iloc[0]
#         new_dataset.at[index, 'mean_temp'] = matching_weather_row['mean_temp']
#         new_dataset.at[index, 'windSpeed'] = matching_weather_row['windSpeed']
#         new_dataset.at[index, 'pressure'] = matching_weather_row['pressure']
#         new_dataset.at[index, 'humidity'] = matching_weather_row['humidity']

# # Print the updated new_dataset
# print(new_dataset)





# Initialize a counter for the number of matches
match_count = 0

# Iterate through rows in new_dataset
for index, row in new_dataset.iterrows():
    if match_count >= 5:
        break

    day = row['day']
    matching_energy_row = energy_daily_selected[energy_daily_selected['day'] == day]
    print('going into loop')

    # If matching energy data is found, display the date and update columns in new_dataset
    if not matching_energy_row.empty:
        match_count += 1
        matching_date = matching_energy_row['day'].iloc[0]
        print(f"Matching date: {matching_date}")

        new_dataset.at[index, 'LCLid'] = matching_energy_row['LCLid'].iloc[0]
        new_dataset.at[index, 'energy_sum'] = matching_energy_row['energy_sum'].iloc[0]
        new_dataset.at[index, 'day'] = matching_energy_row['day'].iloc[0]

# Print the updated new_dataset
print(new_dataset)



going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going into loop
going in

# Splitting dataset to training and testing sets

Available datasets

(for 500+ days)(household nos:5566 (in daily dataset))
Daily bases
Half-Hourly bases
Each household has different amount of data(ie. some have more days of data)

I need to create a code for taking in/loading all the data from the .csv files correctly.

To show that i need to load one household data using an id.

Then i need take 2(to begin with) random household id and club all the data in the id to do that i need to display them as well.



